In [39]:
import requests
import simplejson as json
import numpy as np
import pandas as pd
headers={'token':'KUPgGIeplzQLLJHQzyfFNWfAnpYawKKx'}


In [62]:
Cities = ['Redding','Sacremento','Fresno','Bakersfield','Los Angeles']
StationIDs = ['GHCND:USW00024257','GHCND:USW00023232','GHCND:USW00093193','GHCND:USW00023155','GHCND:USW00023129']

DataCatagories = ['AWND','PRCP','TAVG','TMAX','TMIN']
weather = pd.DataFrame()

for index,StationID in enumerate(StationIDs):
    for DataCatagory in DataCatagories:
        params={'datasetid':'GHCND','stationid':StationID,'limit':'1000','startdate':'2017-01-01','enddate':'2017-12-31','datatypeid':DataCatagory,'units':'standard'}
        test = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data', headers=headers,params=params)
        #print(test.text)

        obj = json.loads(test.text)['results']
        df = pd.DataFrame.from_dict(obj)[['date','value']]
        df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True).dt.date
        df = df.set_index('date')


        params2={'datasetid':'GHCND','stationid':StationID,'limit':'1000','startdate':'2018-01-01','enddate':'2018-12-31','datatypeid':DataCatagory,'units':'standard'}
        test2 = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data', headers=headers,params=params2)
        #print(test.text)
        obj2 = json.loads(test2.text)['results']
        df2 = pd.DataFrame.from_dict(obj2)[['date','value']]
        df2['date'] = pd.to_datetime(df2['date'], infer_datetime_format=True).dt.date
        df2 = df2.set_index('date')
        df = df.append(df2)
        if not weather.empty:
            weather['{} {}'.format(Cities[index],DataCatagory)] = df['value']
        else:
            weather = df.rename(columns={'value':'{} {}'.format(Cities[index],DataCatagory)})

weather.to_pickle('./weather.pkl')